### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [565]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
#%load_ext pycodestyle_magic

In [790]:
# %%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=0, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - np.sum((l_c ** 2) / l_s + (r_c ** 2) / r_s, axis=1) / ((l_s + r_s).flatten())

    def __entropy(self, l_c, l_s, r_c, r_s):
        h_r = - np.sum((r_c / r_s) * np.log(r_c / r_s))
        h_l = - np.sum((l_c / l_s) * np.log(l_c / l_s))
        return (l_s * h_l + r_s * h_r) / (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        h_r = 1 - np.max(r_c / r_s, axis=1)
        l_r = 1 - np.max(l_c / l_s, axis=1)
        return (l_s * h_l + r_s * h_r) / (l_s + r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature) + 1)]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        '''
        В начале сортируем массивы
        После отбрасываем крайние элементы, так как по ним не пройдет разбиение
        (position_changes)После создаем массив индексов, на которых меняются классы в массиве y
        Если в y уже один класс, то выходим
        (eq_el_count)Подсчитаем количество элементов между границами(граница - место, где меняется класс)
        (one_hot_code)Составляем массив размерностью(кол-во изменений на кол-во классов),
        1 помещаем туда на каком классе происходит изменение
        (class_increments) Это массив на базе массива one_hot_code, 
        но в нем уже записаны элементы с учетом их количества, 
        расположенные между двумя возможными соседними точками разделения
        (l_class_count)Массив на базе class_increments,
        в котором уже записаны все элементы до точки разделения(по факту сумма на префиксах)
        (r_class_count)То же что и l_class_count, но уже в обратном порядке(по факту сумма на суффиксах)
        (l_sizes, r_sizes ) Размеры левых(правых) частей при разделении массива по определенной точке
        Далее считаем ошибку и берем минимум
    '''

        x_sort, y_sort = self.__sort_samples(x, y)
        class_number = self.num_class
        bor = int(self.min_samples_split / 2 - 1)
        y_split = y_sort
        if bor:
            y_split = y_sort[bor:-bor]
        position_changes = np.where(y_split[:-1] != y_split[1:])[0] + (bor + 1)

        if len(position_changes) == 0:
            return np.inf, None

        eq_el_count = position_changes - np.append([bor], position_changes[:-1])
        one_hot_code = np.zeros((position_changes.shape[0], class_number))
        one_hot_code[np.arange(position_changes.shape[0]), y_sort[position_changes - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(y_sort[:bor], minlength=class_number)

        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(y_sort, minlength=class_number) - l_class_count
        l_sizes = position_changes.reshape(l_class_count.shape[0], 1)
        r_sizes = y_sort.shape[0] - l_sizes
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        left_el_id = l_sizes[idx][0]
        return gs[idx], (x_sort[left_el_id - 1] + x_sort[left_el_id]) / 2.0

    def Is_leaf(self, y, depth):
        return (0 != self.max_depth <= depth) or \
               (self.sufficient_share <= np.bincount(y).argmax() / y.size) or \
               (y.size < self.min_samples_split)

    def __fit_node(self, x, y, node_id, depth):
        if self.Is_leaf(y, depth):
            self.tree[node_id] = [self.__class__.LEAF_TYPE, np.argmax(np.bincount(y))]
        else:
            thresholds = np.array([self.__find_threshold(x[:, i], y)
                                   for i in self.get_feature_ids(x.shape[1])])
            max_feature = np.argmin(thresholds[:, 0])
            max_threshold = thresholds[max_feature, 1]

            if max_threshold == None:
                self.tree[node_id] = [self.__class__.LEAF_TYPE, np.argmax(np.bincount(y))]
            else:
                l_x, r_x, l_y, r_y = self.__div_samples(x, y, max_feature, max_threshold)

                if (l_x.size == 0) or (r_x.size == 0):
                    self.tree[node_id] = [self.__class__.LEAF_TYPE, np.argmax(np.bincount(y))]
                else:
                    self.tree[node_id] = [self.__class__.NON_LEAF_TYPE, max_feature, max_threshold]
                    self.__fit_node(l_x, l_y, 2 * node_id + 1, depth + 1)
                    self.__fit_node(r_x, r_y, 2 * node_id + 2, depth + 1)
                    l_s = np.array(l_y.size)
                    r_s = np.array(r_y.size)
                    if self.G_function == self.__gini:
                        c = 1 - np.sum((np.unique(y, return_counts=True)[1] / y.size) ** 2)
                        l_c = 1 - np.sum((np.unique(l_y, return_counts=True)[1] / l_y.size) ** 2)
                        r_c = 1 - np.sum((np.unique(r_y, return_counts=True)[1] / r_y.size) ** 2)
                        self.feature_importances_[max_feature] += (y.size * c -
                                                                   l_y.size * l_c - r_y.size * r_c)
                    if self.G_function == self.__entropy:
                        c = -np.sum(np.unique(y, return_counts=True)[1] *
                                    np.log2(np.unique(y, return_counts=True)[1])) / y.size
                        l_c = -np.sum(np.unique(l_y, return_counts=True)[1] *
                                      np.log2(np.unique(l_y, return_counts=True)[1])) / l_y.size
                        r_c = -np.sum(np.unique(r_y, return_counts=True)[1] *
                                      np.log2(np.unique(r_y, return_counts=True)[1])) / r_y.size
                        self.feature_importances_[best_id] += y.size * c - l_y.size * l_c - l_y.size * r_c
                    if self.G_function == self.__misclass:
                        c = 1 - np.max(np.sum(np.unique(y, return_counts=True)[1])) / y.size
                        c_l = 1 - np.max(np.sum(np.unique(l_y, return_counts=True)[1])) / y.size
                        c_r = 1 - np.max(np.sum(np.unique(r_y, return_counts=True)[1])) / y.size
                        self.feature_importances_[best_id] += y.size * c - l_y.size * l_c - r_y.size * r_c

    def fit(self, x, y):
        y = y.astype('int64')
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(len(x[0]))
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= np.sum(self.feature_importances_)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [573]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [574]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)
y_train

array([0, 1, 2, 1, 0, 2, 1, 2, 2, 1, 0, 0, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1,
       0, 1, 2, 2, 0, 1, 0, 2, 1, 2, 0, 2, 0, 0, 2, 1, 1, 2, 1, 0, 1, 2,
       1, 2, 1, 0, 1, 0, 2, 0, 1, 1, 0, 1, 0, 0, 0, 2, 0, 2, 1, 1, 0, 2,
       1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 0, 0, 1, 1, 1, 0, 0, 2, 2,
       1, 2, 2, 0, 0, 2, 2, 0, 1, 0, 1, 0, 0, 2, 2, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 0, 1, 2, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 0, 0, 1, 0, 2, 1, 0, 0, 0, 2,
       2, 2, 0, 2, 0, 0])

## Проверка скорости работы на wine

In [575]:
%time clf.fit(X_train, y_train)

Wall time: 2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [576]:
%time my_clf.fit(X_train, y_train)

Wall time: 9.97 ms


## Проверка качества работы на wine

In [577]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8885003885003885

In [578]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8885003885003885

## Подготовка данных Speed Dating Data 

In [777]:
df = pd.read_csv('Speed Dating Data.csv', encoding='ansi')
for i in df.columns:
    if type(df[i][0]) not in [np.float64, np.int64, float]:
        df.drop(i, axis=1, inplace=True)
for i in df.columns:
    if df[i].isnull().sum() > 500:
        df.drop(i, axis=1, inplace=True)


In [778]:
df = df.fillna(-10000)
df.drop('id', axis=1)
y = df.loc[:, 'gender'].values
X = df.drop('gender', axis=1).iloc[:, :].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [779]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [780]:
%time clf.fit(X_train, y_train)

Wall time: 122 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [781]:
%time my_clf.fit(X_train, y_train)

Wall time: 859 ms


## Проверка качества работы на Speed Dating Data

In [782]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9880619128006781

In [783]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9880606881773744

## Задание 3

In [784]:
data = pd.DataFrame({})
data['my_features'] = np.argsort(my_clf.feature_importances_)[-10:]
data['sklearn_features'] = np.argsort(clf.feature_importances_)[-10:]
data

,my_features,sklearn_features
0,3,35
1,45,45
2,66,60
3,60,53
4,53,0
5,9,66
6,0,9
7,62,62
8,15,15
9,56,56


## Задание 4

In [785]:
estim = RandomForestClassifier(n_estimators=10)
param_grid = {
              'max_features': ['log2', 'sqrt', None],
              'min_samples_split': range(2, 15),
              'max_depth': [i for i in range(5, 20)],
              'criterion': ['gini', 'entropy']
            }
grid_cv = GridSearchCV(estim, param_grid)
grid_cv.fit(X, y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_features': ['log2', 'sqrt', None], 'min_samples_split': range(2, 15), 'max_depth': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [786]:
grid_cv.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=5, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [787]:
grid_cv.best_params_

{'criterion': 'entropy',
 'max_depth': 5,
 'max_features': 'sqrt',
 'min_samples_split': 6}

## Задание 5